In [110]:
from xpinyin import Pinyin
import pandas as pd
from keras.layers import LSTM
from keras.models import Sequential
import numpy as np

In [100]:
p = Pinyin()
df = pd.read_csv('ChineseNames.csv')

In [101]:
#change headers to english
df.columns = ['Name', 'Gender']

In [102]:
#strip last name
def strip_last_name(name):
    return name[1:]

def convert_to_pinyin(name):
    return p.get_pinyin(name)

def process_name(name):
    name = strip_last_name(name)
    name = convert_to_pinyin(name)
    return name

In [103]:
#visualize before after processing
print(df.tail(5))
df['Name'] = df['Name'].apply(process_name)
print(df.tail(5))

     Name Gender
9777   邹敏      F
9778  邹泞憶      F
9779  邹莎莎      F
9780  邹苇航      F
9781  邹雪飞      F
9782  邹雅文      F
9783  邹颜鞠      F
9784  邹一鑫      F
9785  邹逸宁      F
9786   邹莹      F
9787  邹滢曼      F
9788  邹雨宏      F
9789  邹雨寰      F
9790  邹玉洁      F
9791   左曈      F
9792  左婉怡      F
9793  左烜晅      F
9794  左雨晴      F
9795   左越      F
9796  左子烨      F
           Name Gender
9777        min      F
9778    ning-yi      F
9779    sha-sha      F
9780   wei-hang      F
9781    xue-fei      F
9782     ya-wen      F
9783     yan-ju      F
9784     yi-xin      F
9785    yi-ning      F
9786       ying      F
9787   ying-man      F
9788    yu-hong      F
9789    yu-huan      F
9790      yu-ji      F
9791       tong      F
9792     wan-yi      F
9793  xuan-xuan      F
9794    yu-qing      F
9795        yue      F
9796      zi-ye      F


In [104]:
#get all unique chars
characters = {}
for i, name in enumerate(df['Name']):
    for char in name:
        if char in characters:
            characters[char] += 1
        else:
            characters[char] = 1

In [105]:
#find the longest name
maxlen = df['Name'].str.len().max()

In [106]:
#create mapping.  though it would be nice if they mapping is alphabetically ordered, but it doesn't matter mathematically.
idx = 0
for c in characters.keys():
    characters[c] = idx
    idx += 1

In [210]:
#create training example of dimension (example #, timestep, # of features (length of 1-hot vector))
num_examples = df.shape[0]
time_steps = maxlen
num_features = len(characters)
def char_to_vec(char):
    vector = np.zeros((1, num_features), dtype=int)
    idx = characters[char]
    vector[0,idx] = 1
    return vector

def name_to_vec(name):
    example = np.zeros((maxlen, num_features), dtype = int)
    for i in range(maxlen):
        if i < len(name):
            char = name[i]
            vector = char_to_vec(char)
            example[i,:] = vector[0,:]
        else:
            example[i,:] = np.zeros((num_features),dtype=int)
    return example

In [232]:
xxx[1,:,:]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],


In [218]:
#convert all examples to vector format
data = np.empty((0, maxlen, num_features))
ctr = 0
for name in df['Name'].head():
    if ctr % 100 == 0:
        print(ctr)
    ctr += 1
    example = name_to_vec(name)
    example = np.expand_dims(example, 0)
    data = np.concatenate((data, example), axis=0)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700


In [246]:
#convert all examples to vector format
xxx = np.empty((df.shape[0], maxlen, num_features))
ctr = 0
for i, name in enumerate(df['Name']):
    if ctr == 10:
        break
    ctr += 1
    print(name)
    example = name_to_vec(name)
    print(example)
    xxx[i,:,:] = example

chao
[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
fang-zhou
[[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0

,Name,Gender
0,chao,M
1,fang-zhou,M
2,lin-feng,M
3,ran,M
4,tai,M
5,wei-cheng,M
6,yu-sen,M
7,yun-long,M
8,yun-xiang,M
9,zheng,M


In [244]:
xxx[0,:,:]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],


In [13]:
model = Sequential()
model.add(LSTM(1, input_shape=(maxlen, num_features), return_sequences=False)
model.compile(loss='categorical_crossentropy', optimizer='adam')

,Name,Gender
0,yi,F
1,zhang-xi,F
2,jing-wen,F
3,lan,F
4,man,F
5,meng-yao,F
6,qiao,F
7,ran,F
8,yang,F
9,yi,F
